### 4

In [3]:
import cvxpy as cp
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [4]:
def mean_absolute_error(y_true, y_pred):
    mae = 0
    for i in range(len(y_true)):
        mae += abs(y_true[i] - y_pred[i])
    return mae/len(y_true)

In [5]:
df = pd.read_csv("wine+quality/winequality-red.csv", sep=';')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, y_train = X[:1400], y[:1400]
X_test, y_test = X[1400:], y[1400:]

# Standardize the features (x-mean(x))/std(x)
scaler = StandardScaler()
# Use X_train to get the mean and standard deviation
X_train_std = scaler.fit_transform(X_train)
# Use the mean and standard deviation from X_train to standardize X_test
# as having X_train and X_test with same standardization terms makes sense.
X_test_std = scaler.transform(X_test)

def addBiasToSamples(data):
    return np.hstack([data, np.ones((data.shape[0], 1))])
    
X_train_std = addBiasToSamples(X_train_std)
X_test_std = addBiasToSamples(X_test_std)

In [8]:
# Model 1 using least square loss
model1 = LinearRegression().fit(X_train_std, y_train)
y_pred1 = model1.predict(X_test_std)
print(mean_absolute_error(y_test, y_pred1))

0.5329671066366284


In [9]:
# Model 2 using Huber loss
w = cp.Variable(X_train.shape[1])
b = cp.Variable()
r = X_train @ w + b - y_train

obj_fn = cp.sum(cp.huber(r, M=0.5))
model2 = cp.Problem(cp.Minimize(obj_fn))
obj_values = model2.solve()

y_pred2 = X_test @ w.value + b.value
print(mean_absolute_error(y_test, y_pred2))

0.5304108520987253


In [10]:
# Model 3 using hinge loss
w = cp.Variable(X_train.shape[1])
b = cp.Variable()
r = X_train @ w + b - y_train

obj_fn = cp.sum(cp.maximum(0, cp.abs(r) - 1/2 ))
model3 = cp.Problem(cp.Minimize(obj_fn))
obj_values = model3.solve()

y_pred3 = X_test @ w.value + b.value
print(mean_absolute_error(y_test, y_pred3))

0.5481057947573723
